In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

from datetime import datetime, timedelta
from torch_geometric.nn import GATv2Conv, GATConv
from torch_geometric.utils import dense_to_sparse
from torch.distributions import Normal, Laplace, RelaxedOneHotCategorical
from torchdiffeq import odeint  # For continuous-time normalizing flows

from feature.scalers import ranged_scaler
from feature.engineering import *
from CARAT.model_utils import *
from CARAT.model import CausalGraphVAE
from CARAT.components import *
from utils.utils import set_seed, logger

# Torch settings"""
"""torch.use_deterministic_algorithms(False)
torch.backends.cudnn.benchmark = False
torch.autograd.profiler.profile(enabled=False)
torch.autograd.profiler.emit_nvtx(enabled=False)
torch.autograd.set_detect_anomaly(mode=False)

# Environment variables
os.environ['CUBLAS_WORKSPACE_CONFIG'] = '167772160'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = "1"
"""
# Set device
device = torch.device('cpu')#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
print(cats_df.shape)
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

(5000000, 20)


In [3]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [4]:
cats_rows_list = metadata.rows(named=True)
cats_df.shape

(5000000, 21)

In [5]:
cats_df = make_stationary(cats_df)

⚠️ Skipping column timestamp (not a float column)
Column: aimp | Final ADF: -2211.6294 | p-value: 0.0000 | Diffs: 0
Column: amud | Final ADF: -8.0091 | p-value: 0.0000 | Diffs: 0
Column: arnd | Final ADF: -18.1248 | p-value: 0.0000 | Diffs: 0
Column: asin1 | Final ADF: -5.7565 | p-value: 0.0000 | Diffs: 1
Column: asin2 | Final ADF: -39.8976 | p-value: 0.0000 | Diffs: 3
Column: adbr | Final ADF: -111.5305 | p-value: 0.0000 | Diffs: 0
Column: adfl | Final ADF: -217.2031 | p-value: 0.0000 | Diffs: 0
Column: bed1 | Final ADF: -35.0214 | p-value: 0.0000 | Diffs: 0
Column: bed2 | Final ADF: -44.5328 | p-value: 0.0000 | Diffs: 0
Column: bfo1 | Final ADF: -8.5614 | p-value: 0.0000 | Diffs: 0
Column: bfo2 | Final ADF: -13.3015 | p-value: 0.0000 | Diffs: 0
Column: bso1 | Final ADF: -2164.1026 | p-value: 0.0000 | Diffs: 1
Column: bso2 | Final ADF: -13.5707 | p-value: 0.0000 | Diffs: 0
Column: bso3 | Final ADF: -14.7270 | p-value: 0.0000 | Diffs: 0
Column: ced1 | Final ADF: -727.3289 | p-value: 0.

In [6]:
cats_df = cats_df.to_pandas()

In [7]:
metadata['affected'].unique().to_list()

["['cso1']", "['cfo1']", "['ced1']"]

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:06,0.0,0.142857,-0.444480,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000072,-0.507953,-0.716059,0.0,0.100401,-0.186461
2023-01-01 00:00:07,0.0,0.142857,-0.446078,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000083,-0.507953,-0.716059,0.0,0.100408,-0.186406
2023-01-01 00:00:08,0.0,0.142857,-0.447166,0.00002,-8.000267e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000094,-0.507953,-0.716059,0.0,0.100416,-0.186345
2023-01-01 00:00:09,0.0,0.142857,-0.442843,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000104,-0.507953,-0.716059,0.0,0.100426,-0.186278
2023-01-01 00:00:10,0.0,0.142857,-0.441320,0.00002,-8.000045e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000113,-0.507953,-0.716059,0.0,0.100438,-0.186205


In [10]:
cats_df.shape

(4999994, 17)

In [11]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [12]:
test_df = test_df.astype('float32')

In [13]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [14]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

In [15]:
TIME_STEPS = 3
BATCH_SIZE = 10000
hidden_dim = 64
latent_dim = 8
num_nodes = 17

dataset_nominal = TimeSeriesDataset(train_df, device=device, time_steps=TIME_STEPS)
dataloader_nominal = DataLoader(dataset_nominal, batch_size=BATCH_SIZE, shuffle=False, pin_memory=False)

# Initialize model and optimizer
model = CausalGraphVAE(input_dim=train_df.shape[1], hidden_dim=hidden_dim,
                        latent_dim=latent_dim, num_nodes=train_df.shape[1],device=device,
                        time_steps=TIME_STEPS, prior_adj=None)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-5)

# Train on nominal data
print("Pretraining on nominal data...")
model.train_model(dataloader_nominal, optimizer, num_epochs=250, patience=30,rho_max=5,alpha_max=2.5)


Pretraining on nominal data...
Epoch 1: Loss = 20710.2357
Recon Loss =4991.9517, KL Loss = 2.2728, Sparsity Loss = 102.5059, Lagrangian Loss = 2.6903
Epoch 51: Loss = 276.7871
Recon Loss =361.1690, KL Loss = 4.3299, Sparsity Loss = 62.7255, Lagrangian Loss = 1.6437
Early stopping triggered. Last Epoch: 91
Recon Loss =201.5016, KL Loss = 4.4729, Sparsity Loss = 61.2792, Lagrangian Loss = 3.0696


In [16]:
prior_adj = model.causal_graph.adj_mat.clone().detach()
#scaled_prior = scale_tensor(prior_adj)
for i, row in enumerate(prior_adj):
    for j, column in enumerate(row):
        if (j in non_causal_indices) and (i in causal_indices) & (i!=j):
            continue
        else:
            prior_adj[i,j] = 0

In [17]:
pd.DataFrame(prior_adj.cpu().detach().numpy(),index=cols,columns=cols)

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
aimp,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000
amud,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000
arnd,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000
asin1,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000
asin2,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000
adbr,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000
adfl,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000
bed1,0.58257,0.58257,0.500250,0.50025,0.50025,0.500250,0.58257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50025,0.50025,0.500250
bed2,0.58257,0.58257,0.500250,0.50025,0.50025,0.500250,0.58257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50025,0.50025,0.501146
bfo1,0.50025,0.50025,0.582570,0.58257,0.58257,0.582570,0.50025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.58379,0.58257,0.582570


In [18]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]
num_nodes = len(train_df.columns)

new_metadata = []
BATCH_SIZE = 64
hidden_dim=64
latent_dim=8

set_seed()
edge_correct = 0
instantaneous_correct = 0
lagged_correct = 0
counterfactual_correct = 0 
rr_correct = 0
total_correct = 0
total_checked = 0
incorrect = []

for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    logger.info('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]
    print('Anomaly: ' + anomaly)
    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    #start_len = mod_df.shape[0]
    #if start_len >1000:
        #start_len = 1000
    start_len = 50
    start_time = anomaly_time- timedelta(seconds=start_len)
    finish_time = end_time + timedelta(seconds=start_len)
    mod_df = test_df[(test_df.index>= start_time) & (test_df.index<= finish_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]

    """
    FIND THE OPTIMAL NUMBER OF LAGS
    """
    TIME_STEPS = most_frequent(find_optimal_lags_for_dataframe(mod_df))+1

    dataset = TimeSeriesDataset(mod_df,device=device, time_steps=TIME_STEPS)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
    X_data = torch.empty(0,device=device)
    T_data = torch.empty(0,device=device)
    for batch_idx, (X_batch, time_batch) in enumerate(dataloader):
        X_data = torch.cat((X_data[:batch_idx], X_batch, X_data[batch_idx:]))
        T_data = torch.cat((T_data[:batch_idx], time_batch, T_data[batch_idx:]))
    
    fine_tuned = CausalGraphVAE(input_dim=num_nodes, hidden_dim=hidden_dim,
                           latent_dim=latent_dim, num_nodes=num_nodes,device=device,
                           time_steps=TIME_STEPS,prior_adj=prior_adj)
    optimizer = torch.optim.AdamW(fine_tuned.parameters(), lr=0.01, weight_decay=1e-5)  # L2 Regularization

    
    fine_tuned.train_model(dataloader, optimizer, num_epochs=2500, patience=50,BATCH_SIZE=BATCH_SIZE,rho_max=5.0,alpha_max=2.5)

    causes = fine_tuned.infer_causal_effect(X_data.to(torch.float32).to(device),T_data.to(torch.float32).to(device),anomaly,cols,non_causal_indices=non_causal_indices)
    
    causes = causes.filter(items=potential_causes, axis=0)
    edge_cause_1 = causes.sort_values(by='causes',ascending=False)[0:3].index[0]
    edge_cause_2 = causes.sort_values(by='causes',ascending=False)[0:3].index[1]
    edge_cause_3 = causes.sort_values(by='causes',ascending=False)[0:3].index[2]
    
    instant_cause_1 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[0]
    instant_cause_2 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[1]
    instant_cause_3 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[2]
    
    lag_cause_1 = causes.sort_values(by='lagged',ascending=False)[0:3].index[0]
    lag_cause_2 = causes.sort_values(by='lagged',ascending=False)[0:3].index[1]
    lag_cause_3 = causes.sort_values(by='lagged',ascending=False)[0:3].index[2]
    
    counterfactual_cause_1 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[0]
    counterfactual_cause_2 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[1]
    counterfactual_cause_3 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[2]

    rr_cause_1 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[0]
    rr_cause_2 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[1]
    rr_cause_3 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[2]
    
    total_score_cause_1=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[0]
    total_score_cause_2=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[1]
    total_score_cause_3=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[2]
    
    if root_cause == edge_cause_1:
        row['edge_cause_1'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_2'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_3'] = 1
    
    if root_cause == counterfactual_cause_1:
        row['counterfactual_cause_1'] = 1
    if root_cause == counterfactual_cause_2:
        row['counterfactual_cause_2'] = 1
    if root_cause == counterfactual_cause_3:
        row['counterfactual_cause_3'] = 1
    
    if root_cause == total_score_cause_1:
        row['total_score_cause_1'] = 1
    if root_cause == total_score_cause_2:
        row['total_score_cause_2'] = 1
    if root_cause == total_score_cause_3:
        row['total_score_cause_3'] = 1
    
    if root_cause == instant_cause_1:
        row['instant_cause_1'] = 1
    if root_cause == instant_cause_2:
        row['instant_cause_2'] = 1
    if root_cause == instant_cause_3:
        row['instant_cause_3'] = 1
    
    if root_cause == lag_cause_1:
        row['lag_cause_1'] = 1
    if root_cause == lag_cause_2:
        row['lag_cause_2'] = 1
    if root_cause == lag_cause_3:
        row['lag_cause_3'] = 1

    if root_cause == rr_cause_1:
        row['rr_cause_1'] = 1
    if root_cause == rr_cause_2:
        row['rr_cause_2'] = 1
    if root_cause == rr_cause_3:
        row['rr_cause_3'] = 1
    
    new_metadata.append(row)
    
    if root_cause in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3]:
        total_correct+=1
    if root_cause in [edge_cause_1 , edge_cause_2 , edge_cause_3]:
        edge_correct+=1
    if root_cause in [counterfactual_cause_1 , counterfactual_cause_2 , counterfactual_cause_3]:
        counterfactual_correct+=1
    if root_cause in [instant_cause_1 , instant_cause_2 , instant_cause_3]:
        instantaneous_correct+=1
    if root_cause in [lag_cause_1 , lag_cause_2 , lag_cause_3]:
        lagged_correct+=1
    if root_cause in [rr_cause_1 , rr_cause_2 , rr_cause_3]:
        rr_correct+=1
    
    total_accuracy = total_correct/total_checked* 100
    edge_accuracy = edge_correct/total_checked* 100
    cf_accuracy = counterfactual_correct/total_checked* 100
    instant_accuracy = instantaneous_correct/total_checked* 100
    lag_accuracy = lagged_correct/total_checked* 100
    rr_accuracy = rr_correct/total_checked* 100
    
    if root_cause not in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3,edge_cause_1 , edge_cause_2 , edge_cause_3 ]:
        incorrect.append(i) 
    logger.info(f"Edge Accuracy = {edge_accuracy:.2f}, Instantaneous Accuracy = {instant_accuracy:.2f}, Lagged Accuracy = {lag_accuracy:.2f}, Counterfactual Accuracy = {cf_accuracy:.2f},  Blended Accuracy = {total_accuracy:.2f},  RR Accuracy = {rr_accuracy:.2f}  ") 



2025-03-16 22:55:28,225 INFO -- Model: 0


Anomaly: cfo1


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [3, 64] but got: [2, 64].